**regression** - fitting a model to data.  The goal is to describe the relationship between the *dependent variables* and the *explanatory variables*.

**multiple regression** multiple explanatory variables.

**linear regression** wen the relationship is linear.
$$
y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \epsilon
$$
where $\beta_0$ is the intercept, $\beta_1$ is the parameter association with $x_1$, $\beta_2$ is the parameter associated with $x_2$ and $\epsilon$ is the residual due to random variation or unknown factors.

**ordinary least squares**.  Given a sequence of values for y and sequences for $x_1$ and $x_2$, we can find the beta parameters that minimize $epsilon^2$

In [7]:
live, firsts, others = first.MakeFrames()
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()
regression.SummarizeResults(results)

Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405


In [9]:
##Results are also available as parameters:
inter = results.params['Intercept']
slope = results.params['agepreg']
slope_pvalue = results.pvalues['agepreg']
results.rsquared

0.0047381154747098142

In [11]:
##this gives the p-value associated with the model as a whole
results.f_pvalue

5.7229471072677547e-11

In [13]:
residuals = results.resid

##this returns a sequence of values corresponding
##to agepreg.
fitted_values = results.fittedvalues

# results.summary() provides a lot of info
#the following is easier:
regression.SummarizeResults(results)

Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405


**spurious** - result for which there is no obvious mechanism that would explain it. e.g. why would first babies be lighter than others?  Perhaps because mothers of first babies are younger...

In [14]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

In [17]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

In [19]:
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']
slope

0.017453851471802891

In [20]:
##expected difference in birthweight for
##first babies and others, due to mother's age:
slope * diff_age

-0.062597099721694888

In [26]:
##this only accounts for half of the observed difference.
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()

regression.SummarizeResults(results)
print 

formula = 'totalwgt_lb ~ agepreg'
results = smf.ols(formula, data=live).fit()

regression.SummarizeResults(results)

Intercept   7.33   (0)
isfirst[T.True]   -0.125   (2.55e-05)
R^2 0.00196
Std(ys) 1.408
Std(res) 1.407

Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405


`isfirst` is a **categorical variable**, i.e. it falls into categories like True or False.  The estimated parameter is the effect on birth weight when `isfirst` is true.  So the result, -0.125 lbs is the difference in birth weight between first babies and others.

In [28]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
regression.SummarizeResults(results)

Intercept   6.91   (0)
isfirst[T.True]   -0.0698   (0.0253)
agepreg   0.0154   (3.93e-08)
R^2 0.005289
Std(ys) 1.408
Std(res) 1.405


In the combined model, the parameter `isfirst` is smaller by about half, which means that part of the apparent effict is accounted for by `agepreg`.  Also, notice the p-value is now on the border of statistical significance.

could the contribution of agepreg be nonlinear? (Note, this is still linear regression, because the explanatory variable is now just the result of a nonlinear function)

In [30]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
regression.SummarizeResults(results)

Intercept   5.69   (1.38e-86)
isfirst[T.True]   -0.0504   (0.109)
agepreg   0.112   (3.23e-07)
agepreg2   -0.00185   (8.8e-06)
R^2 0.007462
Std(ys) 1.408
Std(res) 1.403


Negative coefficient in agepreg2 corroborates with downward curving of figure 10.2

in this example, the mother's age acts as a **control variable** for the diference in age between first-time mothers and others, making it possible to isolate the effect of `isfirst`

In [46]:
live = live[live.prglngth>30]
resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [50]:
t = []
for name in join.columns:

    try:
        if join[name].var() < 1e-7:
            continue
            
        formula = 'totalwgt_lb ~ agepreg + ' + str(name)
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2: #num of observations
            continue
        
        results = model.fit()
    except(ValueError, TypeError):
        continue
        
    t.append((results.rsquared, name))
t

[(0.0053576473236404132, u'caseid'),
 (0.0057500139850769072, u'pregordr'),
 (0.0063309802373898716, u'pregend1'),
 (0.016017752709788002, u'nbrnaliv'),
 (0.0055431561930947559, u'cmprgend'),
 (0.0054428005916395961, u'cmprgbeg'),
 (0.0053276126015608938, u'gestasun_m'),
 (0.0070235526384532232, u'gestasun_w'),
 (0.1234004136336101, u'wksgest'),
 (0.027144274639579802, u'mosgest'),
 (0.0053368691675184099, u'bpa_bdscheck1'),
 (0.018550925293941867, u'babysex'),
 (0.94981273059780091, u'birthwgt_lb'),
 (0.013102457615706053, u'birthwgt_oz'),
 (0.0055431561930947559, u'cmbabdob'),
 (0.0056849526500283298, u'kidage'),
 (0.006165319836040517, u'hpagelb'),
 (0.0080663173686768008, u'matchfound'),
 (0.012529022541810764, u'anynurse'),
 (0.0044098205836260451, u'frsteatd_n'),
 (0.0042639734717098143, u'frsteatd_p'),
 (0.00402013146273561, u'frsteatd'),
 (0.0058305717702541449, u'cmlastlb'),
 (0.0053567472661234516, u'cmfstprg'),
 (0.0054283336509906022, u'cmlstprg'),
 (0.0057314017337591894, 

In [52]:
t.sort(reverse=True)
for mse, name in t[:30]:
    print name, mse

totalwgt_lb 1.0
birthwgt_lb 0.949812730598
lbw1 0.300824078447
prglngth 0.130125194886
wksgest 0.123400413634
agecon 0.102031499282
mosgest 0.0271442746396
babysex 0.0185509252939
race_r 0.0161995035863
race 0.0161995035863
nbrnaliv 0.0160177527098
paydu 0.0140037955781
rmarout03 0.0134300664657
birthwgt_oz 0.0131024576157
anynurse 0.0125290225418
bfeedwks 0.0121936884045
totincr 0.0118700690312
marout03 0.0118078019944
marcon03 0.0117525993544
cebow 0.0114377709196
rmarout01 0.0114077371386
rmarout6 0.0113541384728
marout01 0.0112693572468
hisprace_r 0.011238349302
hisprace 0.011238349302
mar1diss 0.0109615635908
fmarcon5 0.0106049646843
rmarout02 0.0105469132066
marcon02 0.0104814017955
fmarout5 0.0104616913674


race is a **proxy variable**; that is apparent correlations are often caused, at least in part, by other factors.

**data mining**
    Sometimes you start with a theory and use data to test it.  Other times you start with data and go looking for possible theories.

In [54]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
           'nbrnaliv > 1 +paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()

In [56]:
regression.SummarizeResults(results)

Intercept   6.63   (0)
C(race)[T.2]   0.357   (5.43e-29)
C(race)[T.3]   0.266   (2.33e-07)
babysex == 1[T.True]   0.295   (5.39e-29)
nbrnaliv > 1[T.True]   -1.38   (5.1e-37)
paydu == 1[T.True]   0.12   (0.000114)
agepreg   0.00741   (0.0035)
totincr   0.0122   (0.00188)
R^2 0.05999
Std(ys) 1.271
Std(res) 1.232


`C(race)` tells Patsy to treat race as a categorical variable

**Poisson regression** if the dependent variable is an integer count.

**logisitc regression** when the dependet variable is boolean.  Expresses predictions in **odds** rather than probabilities--i.e. the ratio of the probability that it will occur to the probability that it will not.

$$
\text{odds} = p / (1 - p)
$$

$$
\text{probability} =  o / (o + 1)
$$

Logistic regression is based on the following model:

$$
\log o = \beta_0 + \beta_1x_1 + \beta_2x_2 + \epsilon
$$

logistic regression does not have a closed form solution, so it is solved by guessing an initial solution and improving it iteratively--looking for MLE, or the set of parameters that maximize the likelyhood of the data.




In [65]:
y = np.array([0,1,0,1])
x1 = np.array([0,0,0,1])
x2 = np.array([0,1,1,1])

beta = [-1.5,2.8,1.1]

log_o = beta[0] + beta[1]*x1 + beta[2]*x2
print "log_o", log_o
o = np.exp(log_o)
print "o", o
p = o / (o + 1)
print "p", p

log_o [-1.5 -0.4 -0.4  2.4]
o [  0.22313016   0.67032005   0.67032005  11.02317638]
p [ 0.18242552  0.40131234  0.40131234  0.9168273 ]


In [66]:
likes = y * p + (1-y) * (1-p)
print "likes", likes
like = np.prod(likes)
print "overall likelihood", like

likes [ 0.81757448  0.40131234  0.59868766  0.9168273 ]
overall likelihood 0.180093352967


Probability is p when y == 1, 1-p when y == 0.  The goal is to maximize the **likelihood** of the data.  

**endogenous variable** - dependent variable

**exogenous** - explanatory



In [67]:
live, first, others = first.MakeFrames()
df = live[live.prglngth>30]

In [70]:
df['boy'] = (df.babysex==1).astype(int)
model = smf.logit('boy ~ agepreg', data=df)
results = model.fit()
regression.SummarizeResults(results)

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Intercept   0.00579   (0.953)
agepreg   0.00105   (0.783)
R^2 6.144e-06


/Users/davidgoldberg/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [85]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=df)
results = model.fit()
regression.SummarizeResults(results)

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3
Intercept   -0.0301   (0.772)
C(race)[T.2]   -0.0224   (0.66)
C(race)[T.3]   -0.000457   (0.996)
agepreg   -0.00267   (0.629)
hpagelb   0.0047   (0.266)
birthord   0.00501   (0.821)
R^2 0.000144


In [1]:
##these dataframes contain the variables
endog = pandas.DataFrame(model.endog, columns=[model.endog_names])
exog = pandas.DataFrame(model.exog, columns=model.exog_names)
exog.head(5)

NameError: name 'pandas' is not defined

###Measuring Accuracy

In [90]:
actual = endog['boy']
baseline = actual.mean()
#this is the baseline prediction if you were to grab
#a random record from the sample
baseline

0.507173764518333

In [91]:
predict = (results.predict() >= 0.5)

true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
##accuracy is a fraction of correct guesses
acc =  (sum(true_pos) + sum(true_neg)) /  len(actual)
print acc

0.511500797085


In [98]:
columns = ['agepreg', 'hpagelb','birthord', 'race']
new = pandas.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
y

array([ 0.5130905])

##Exercises

####exercise 11.1

In [142]:
df = join[join.prglngth > 30]
def datamine(df, dep_variable):
    """
    expl_variables:  a list of explanatory variables to
        be considered in the mining.
    """
    t = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue
            
            formula = str(dep_variable) + " ~ " + str(name)
            model = smf.ols(formula, data=df)
            if model.nobs < len(df) / 2:
                continue
                
            results = model.fit()
        except (ValueError, TypeError):
            continue
        
        t.append((name,results))
    return t
   
t = datamine(df, 'prglngth')

In [181]:
def SortAndPrint(t):
    """
    t is a list of tuples with regression results 
       in the form (name of parameter, results object)
    """
    t.sort(key=lambda x: x[1].rsquared, reverse=True)
    for name, res in t:
        try:
            if res.pvalues[name] < 0.1:
                print name, res.params[name], res.rsquared
        except KeyError, e:
            print "EXCEPTION, ",e
SortAndPrint(t)

prglngth 1.0 1.0
wksgest 0.823927251016 0.806243411614
totalwgt_lb 0.528064784847 0.124457431481
birthwgt_lb 0.514962860811 0.119773078049
lbw1 2.48648368743 0.103725422046
mosgest 0.417548478248 0.0956243198959
prglngth_i -1.77679367574 0.0220537757965
canhaver 0.11831014212 0.00605049526819
datcon01_i -0.47208859129 0.00581775529988
con1mar1_i -0.443886529558 0.00554637613623
nbrnaliv -0.727471007121 0.00457756578553
mar1con1_i -0.401666091852 0.00315080225387
anynurse -0.0479292615804 0.00245202488371
bfeedwks -0.000191129876341 0.00236918394467
pregend1 0.213292014116 0.00224938943379
rmarout11_i -2.68111336488 0.0022436279681
marout11_i -2.68111336488 0.0022436279681
marcon11_i -2.68111336488 0.0022436279681
cmlastlb_r -0.000259061163168 0.0020431424422
cmlastlb -0.000259061163168 0.0020431424422
datend02_i -1.07513070601 0.00201248339275
datcon02_i -1.07513070601 0.00201248339275
agecon02_i -0.99305345975 0.00198828676889
fmarcon5_i -0.326061018289 0.00196815932425
ageprg02_i -0.

In [179]:
model = smf.ols('prglngth ~ birthord + C(race) + C(paydu) + + C(diabetes)', data=df)
results = model.fit()
regression.SummarizeResults(results)

Intercept   38.7   (0)
C(race)[T.2]   0.0984   (0.0417)
C(race)[T.3]   0.00545   (0.944)
C(paydu)[T.2]   -0.0763   (0.0678)
C(paydu)[T.8]   -1.39   (0.028)
C(paydu)[T.9]   -1.92   (0.0236)
C(diabetes)[T.5]   0.273   (0.000327)
C(diabetes)[T.8]   1.01   (0.341)
C(diabetes)[T.9]   0.317   (0.71)
birthord   -0.0535   (0.00596)
R^2 0.005232
Std(ys) 1.898
Std(res) 1.893


##Exercise 11.2

In [210]:
def GoMiningBabySex(df):
    df.dropna(subset=['babysex'])
    df['boy'] = (df.babysex==1).astype(int)
    
    t = []
    
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue
            
            formula = "boy ~ " + str(name)
            model = smf.logit(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df) / 2:
                continue
                
            results = model.fit()
        except:
            continue
        
        t.append((results.prsquared, name))
    return t

bs = GoMiningBabySex(df)


Optimization terminated successfully.
         Current function value: 0.692996
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692962
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692850
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693001
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692903
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692766
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692995
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693017
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692989
  

/Users/davidgoldberg/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/davidgoldberg/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)



         Current function value: 0.692967
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692987
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692978
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692964
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693010
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692999
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693005
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693018
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692960
         Iterations 3
Optimization te

/Users/davidgoldberg/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/davidgoldberg/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)



         Current function value: 0.692943
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692832
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693008
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692795
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692693
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692460
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692822
         Iterations 4
Optimization te

In [211]:
regression.MiningReport(bs)

totalwgt_lb 0.00967200535184
birthwgt_lb 0.0092489816712 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
constat3 0.0010867056923 3RD PRIORITY CODE FOR CURRENT CONTRACEPTIVE STATUS
lbw1 0.00104859533196 LOW BIRTHWEIGHT - BABY 1
nplaced 0.00100691837365 # OF R'S BIO CHILDREN SHE PLACED FOR ADOPTION (BASED ON BPA)
infever 0.000806910464076 EVER USED INFERTILITY SERVICES OF ANY KIND
frsteatd 0.0007600842059 AGE (IN MOS) WHEN 1ST SUPPLEMENTED - 1ST FROM THIS PREG
splstwk1 0.000731746186575 IF-1 H/P DOING WHAT LAST WEEK (EMPLOYMENT STATUS) 1ST MENTION
outcom02 0.000698741289162 OUTCOME OF PREGNANCY - 2ND
fmarout5 0.000681996122597 FORMAL MARITAL STATUS AT PREGNANCY OUTCOME
nummult34 0.000656746961037 NUMBER OF METHODS REPORTED IN (OCT 2001)
coh1dur 0.000654992932551 DURATION (IN MONTHS) OF R'S FIRST COHABITATION
brnout_r 0.000641068785277 IB-8 R BORN OUTSIDE OF US
brnout 0.000641068785277 IB-8 R BORN OUTSIDE OF US
bpa_bdscheck1 0.000635132392883 WHETHER 1ST LIVEBORN BABY FROM THIS

In [221]:
model = smf.logit('boy ~ fmarout5==5 + totalwgt_lb + nplaced + infever==1', data=df)
results = model.fit()
regression.SummarizeResults(results)

Optimization terminated successfully.
         Current function value: 0.683971
         Iterations 6
Intercept   -1.5   (4.32e-29)
fmarout5 == 5[T.True]   0.199   (1.11e-05)
infever == 1[T.True]   0.26   (7.28e-05)
totalwgt_lb   0.193   (1.55e-28)
nplaced   -1.78   (0.00491)
R^2 0.01307


###Exercise 11.3

Use a **Poisson Regression** to predict how many children a woman has born if she is 35 years old, black, college graduate, with household income greater than 75000

In [222]:
def makeJoin():
    live, firsts, others = first.MakeFrames()
    live = live[live.prglngth>30]
    resp = chap01soln.ReadFemResp()
    resp.index = resp.caseid
    join = live.join(resp, on='caseid', rsuffix='_r')
    return join

join = makeJoin()

In [259]:
join = join.dropna(subset=['agepreg'])
join.numbabes.replace([97], np.nan, inplace=True)
join['is35'] = ((join.agepreg >= 35) * (join.agepreg < 36)).astype(int)
formula = 'numbabes ~ C(race) + age_r + C(havedeg) + C(totinc)'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.586266
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 4239
Model:                        Poisson   Df Residuals:                     4218
Method:                           MLE   Df Model:                           20
Date:                Wed, 22 Jul 2015   Pseudo R-squ.:                 0.01613
Time:                        12:43:50   Log-Likelihood:                -6724.2
converged:                       True   LL-Null:                       -6834.4
                                        LLR p-value:                 9.888e-36
=====================================================================================
                        coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept             0.1345      0.100      1.345      0.179        -0.062     0.331
C(race)[T.2]         -0.0509      0.024     -2.082      0.037        -0.099    -0.003
C(race)[T.3]         -0.0722      0.041     -1.747      0.081        -0.153     0.009
C(havedeg)[T.5.0]     0.0497      0.021      2.334      0.020         0.008     0.091
C(totinc)[T.2]        0.0624      0.102      0.610      0.542        -0.138     0.263
C(totinc)[T.3]        0.2575      0.096      2.671      0.008         0.069     0.447
C(totinc)[T.4]        0.3055      0.090      3.389      0.001         0.129     0.482
C(totinc)[T.5]        0.1083      0.096      1.122      0.262        -0.081     0.297
C(totinc)[T.6]        0.1836      0.086      2.131      0.033         0.015     0.352
C(totinc)[T.7]        0.0953      0.087      1.098      0.272        -0.075     0.266
C(totinc)[T.8]        0.0441      0.087      0.508      0.612        -0.126     0.215
C(totinc)[T.9]        0.0146      0.088      0.166      0.868        -0.157     0.186
C(totinc)[T.10]       0.0197      0.088      0.223      0.824        -0.154     0.193
C(totinc)[T.11]      -0.0378      0.086     -0.440      0.660        -0.206     0.131
C(totinc)[T.12]      -0.0291      0.085     -0.341      0.733        -0.196     0.138
C(totinc)[T.13]      -0.0168      0.085     -0.197      0.844        -0.184     0.150
C(totinc)[T.14]      -0.0394      0.082     -0.481      0.630        -0.200     0.121
C(totinc)[T.97]       0.1448      0.288      0.502      0.616        -0.420     0.710
C(totinc)[T.98]      -0.0501      0.099     -0.507      0.612        -0.244     0.144
C(totinc)[T.99]       0.0711      0.131      0.545      0.586        -0.185     0.327
age_r                 0.0208      0.002     12.038      0.000         0.017     0.024
=====================================================================================
"""

In [260]:
columns = ['race', 'age_r', 'havedeg', 'totinc']
new = pandas.DataFrame([[1, 35, 1, 14]], columns=columns)
y = results.predict(new)
y


array([ 2.27943811])

###Exercise 11.4

use **multinomial logistic regression** to determine the categorical marital status of a 25 year old white high school graduate whose annual income is about $45,000



In [273]:
resp = chap01soln.ReadFemResp()

formula = 'rmarital ~ age_r + C(race) + C(educat) + C(totincr)'
model = smf.mnlogit(formula, data=resp)
results = model.fit()
results.summary()

         Current function value: 1.054219
         Iterations: 35


/Users/davidgoldberg/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 7643
Model:                        MNLogit   Df Residuals:                     7508
Method:                           MLE   Df Model:                          130
Date:                Wed, 22 Jul 2015   Pseudo R-squ.:                  0.1906
Time:                        13:06:13   Log-Likelihood:                -8057.4
converged:                      False   LL-Null:                       -9954.7
                                        LLR p-value:                     0.000
====================================================================================
      rmarital=2       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept            2.5873      0.319      8.102      0.000         1.961     3.213
C(race)[T.2]        -0.6489      0.112     -5.790      0.000        -0.869    -0.429
C(race)[T.3]        -0.7345      0.173     -4.248      0.000        -1.073    -0.396
C(educat)[T.10]      0.3770      0.197      1.915      0.055        -0.009     0.763
C(educat)[T.11]     -0.2196      0.210     -1.044      0.296        -0.632     0.193
C(educat)[T.12]     -0.1215      0.138     -0.881      0.378        -0.392     0.149
C(educat)[T.13]     -0.2167      0.172     -1.262      0.207        -0.553     0.120
C(educat)[T.14]     -0.0890      0.168     -0.529      0.596        -0.419     0.241
C(educat)[T.15]     -0.3148      0.214     -1.469      0.142        -0.735     0.105
C(educat)[T.16]     -0.7445      0.195     -3.812      0.000        -1.127    -0.362
C(educat)[T.17]     -0.5642      0.273     -2.068      0.039        -1.099    -0.030
C(educat)[T.18]     -1.1754      0.409     -2.872      0.004        -1.977    -0.373
C(educat)[T.19]      0.1120      0.288      0.389      0.697        -0.452     0.676
C(totincr)[T.2]      0.5144      0.331      1.553      0.121        -0.135     1.164
C(totincr)[T.3]      0.1893      0.341      0.555      0.579        -0.480     0.858
C(totincr)[T.4]     -0.1125      0.308     -0.365      0.715        -0.716     0.491
C(totincr)[T.5]     -0.1790      0.287     -0.624      0.533        -0.742     0.383
C(totincr)[T.6]     -0.3620      0.281     -1.290      0.197        -0.912     0.188
C(totincr)[T.7]     -0.3301      0.279     -1.184      0.236        -0.876     0.216
C(totincr)[T.8]     -0.3530      0.276     -1.279      0.201        -0.894     0.188
C(totincr)[T.9]     -0.5145      0.280     -1.837      0.066        -1.064     0.034
C(totincr)[T.10]    -0.4745      0.290     -1.635      0.102        -1.043     0.094
C(totincr)[T.11]    -0.8088      0.282     -2.873      0.004        -1.361    -0.257
C(totincr)[T.12]    -1.1122      0.288     -3.860      0.000        -1.677    -0.547
C(totincr)[T.13]    -1.1297      0.294     -3.838      0.000        -1.707    -0.553
C(totincr)[T.14]    -1.3146      0.279     -4.711      0.000        -1.862    -0.768
age_r               -0.0864      0.006    -14.865      0.000        -0.098    -0.075
------------------------------------------------------------------------------------
      rmarital=3       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -7.4010      1.299     -5.697      0.000        -9.947    -4.855
C(race)[T.2]        -0.1220      0.366     -0.333      0.739        -0.840     0.596
C(race)[T.3]         0.4449      0.514      0.865      0.387        -0.563     1.453
C(educat)[T.10]      0.3587      0.699      0.513      0.608        -1.011     1.729
C(educat)[T.11]      1.1767      0.514      2.288      0.022         0.169     2.185
C(educat)[T

In [275]:
columns = ['age_r','race','educat','totincr']
new = pandas.DataFrame([[25, 2, 12, 11]], columns=columns)
y = results.predict(new)
y

array([[ 0.43494745,  0.13738324,  0.00044677,  0.02698496,  0.0170361 ,
         0.38320148]])

43% are currently married
14% are not married but currently living with opp sex partner
An extremely small amount are widowed
2.6% are divorced
1.7% are separated due to marital discord
38% have never been maried.

In [281]:
columns = ['age_r','race','educat','totincr']
new = pandas.DataFrame([[43, 2, 16, 14]], columns=columns)
y = results.predict(new)
y

array([[ 0.86989957,  0.01875586,  0.0009998 ,  0.06387563,  0.01264673,
         0.0338224 ]])